# Moderacion de contenido

## Moderacion de contenido en imagenes

In [ ]:
# Iniciamos las librerias
import boto3
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
# Iniciamos el cliente
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
# Colocamos el nombre de nuestro bucket
bucketName = "{REPLACE_YOUR_BUCKET}"
imageName = "moderacion-de-contenido/yoga_swimwear.jpg"

In [ ]:
# Subimos la imagen a S3
s3.upload_file(
    Filename="recursos/yoga_swimwear.jpg",
    Bucket=bucketName,
    Key=imageName,
)

In [ ]:
# Mostramos la imagen desde S3
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

In [ ]:
# Llamamos a la API de rekognition para detectar el contenido inapropiado
detectModerationLabelsResponse = rekognition.detect_moderation_labels(
   Image={
       'S3Object': {
           'Bucket': bucketName,
           'Name': imageName,
       }
   }
   #,MinConfidence=80
)

In [ ]:
# Mostramos el JSON retornado por la API
display(detectModerationLabelsResponse)

In [ ]:
# Mostramos la lista de etiquetas detectadas
for label in detectModerationLabelsResponse["ModerationLabels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
    print("  - Parent: {}".format(label["ParentName"]))

## Moderacion de contenido en videos

In [ ]:
# Direccion donde subiremos nuestro video a S3
videoName = "moderacion-de-contenido/moderation.mov"

In [ ]:
# Subimos la imagen a S3
s3.upload_file(
    Filename="recursos/moderation.mov",
    Bucket=bucketName,
    Key=videoName,
)

In [ ]:
# Empezamos el job de moderacion de contenido
startModerationLabelDetection = rekognition.start_content_moderation(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

moderationJobId = startModerationLabelDetection['JobId']
display("Job Id: {0}".format(moderationJobId))

In [ ]:
# Esperamos que el job de moderacion de contenido termine
getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP'
)

while(getContentModeration['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP')
    
display(getContentModeration['JobStatus'])

In [ ]:
# Mostramos el JSON retornado por la API
display(getContentModeration)

In [ ]:
theObjects = {}

strDetail = "Moderation labels in video<br>=======================================<br>"
strOverall = "Moderation labels in the overall video:<br>=======================================<br>"

# Contenido inseguro detectado en cada frame
for obj in getContentModeration['ModerationLabels']:
    ts = obj ["Timestamp"]
    cconfidence = obj['ModerationLabel']["Confidence"]
    oname = obj['ModerationLabel']["Name"]
    strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, oname, round(cconfidence,2))
    if oname in theObjects:
        cojb = theObjects[oname]
        theObjects[oname] = {"Name" : oname, "Count": 1+cojb["Count"]}
    else:
        theObjects[oname] = {"Name" : oname, "Count": 1}

# Objetos unicos detectado en el video
for theObject in theObjects:
    strOverall = strOverall + "Name: {}, Count: {}<br>".format(theObject, theObjects[theObject]["Count"])

# Mostramos los resultados
display(HTML(strOverall))

In [ ]:
# Mostramos el video
s3VideoUrl = s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': videoName})
videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)
videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)
display(HTML(videoui))

In [ ]:
listui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(strDetail)
display(HTML(listui))